# 1.OpenAPI을 이용하여 데이터 수집

In [18]:
import requests
import pandas as pd
import json

#1. 타겟 사이트 url
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2023-01-01&endDt=2023-12-30&age=20&authKey="

# 2. 페이지 로딩
res = requests.get(url)

if res.status_code == 200:
    print("Request successful")
else:
    print("Error:", res.status_code)
    
print(res.text)

# # Decode binary content to UTF-8 string
# content_str = res.content.decode('utf-8')

# 3. Parse the JSON data
data = json.loads(res.text)
# data = json.loads(content_str)


# 4. Print the parsed JSON data
print(data)

# 5. ETL
# 5-1 데이터 추출
books = []
for d in data['response']['docs']:
    books.append(d['doc'])

# books = [d['doc'] for d in data['response']['docs']] # list complrehension

# 5-2 리스트을 이용하여  데이터프레임 생성:  list -> DataFrame
books_df = pd.DataFrame(books)

books_df.head()

# 5-3 dataFrame 객체을 json 파일로 저장
books_df.to_json('20s_best_book_2023.json')



Request successful
{"response":{"error":"인증정보가 일치하지 않습니다."}}
{'response': {'error': '인증정보가 일치하지 않습니다.'}}


KeyError: 'docs'

## 2. Open API을 활용한 데이터셋 가공 작업 :  "20대가 가장 좋아하는 도서목록"

In [ ]:
# 1. json file 로딩 -> dataFrame 객체 변환
books_df = pd.read_json('20s_best_book_2023.json')
books_df.head()

In [ ]:
# 2. 특정 열을 선택하여 새 데이터프레임 생성 - 컬럼 인덱싱 이용
books_df.columns
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']]
books.head()

In [ ]:
# method) 원하는 데이터프레임 행과 열 동시에 슬라이싱으로 선택하기 - df.loc[:,:]
books = books_df.loc[:, 'no':'isbn13']
books.head()

## 3. 추가 데이터 수집: 각 도서별 페이지수 찾기  
    *  HTML에서 데이터 추출하기 - BeautifulSoup
    1. 크롬 브라우저 개발자 도구로 HTML 태그 찾기
    2. 태그 위치 찾기 : soup.find('찾을 태그 이름',attrs={'태그 속성을 딕셔너리로 지정'} 
    3. soup.select()
    

### [문제1] BeautifulSoup으로 HTML에서 첫 번째 도서명의 링크 주소 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Search?query=9791189327156 # '물고기는 존재하지 않는다'의 ISBN

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
isbn = 9791189327156
# 1. target url
url = 'https://www.yes24.com/Product/Search?query={}'
r = requests.get(url.format(isbn))
# 2. response message -> soup object
soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)
#print(prd_link['href'])

### [문제2] BeautifulSoup으로 HTML에서 첫 번째 도서의 페이지수가 담긴 HTML의 위치 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Goods/105526047
* soup.select()

In [ ]:
# soup.select()
url = 'https://www.yes24.com/Product/Goods/105526047'
r= requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
print(prd_detail) # list

# if prd_detail is not None:
#     page = prd_detail[0].text.split('|')[0]
#     print(page)
# else:
#     print("The element you're trying to select doesn't exist.")


### [문제3]  20대가 선호하는 전체 도서의 쪽수 구하여 Series 만들기

In [ ]:
#1. 사용자 정의 함수 : isbn을 전달받아 해당 도서의 페이지 수 반환하기
def get_page_cnt(isbn):
    # 1. Yes24 도서 검색 페이지 URL에 ISBN을 넣어 HTML 가져오기.
    url = 'https://www.yes24.com/Product/Search?query={}'
    r = requests.get(url.format(isbn))
    #2. soup 객체로 변환
    soup = BeautifulSoup(r.text, 'html.parser')   
    # 3.검색 결과에서 해당 도서를 선택하기.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 4. 도서 상세 페이지를 가져오기.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 5. 쪽수가 들어 있는 td를 찾기.
    prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
    print(prd_detail)
    # 6. 페이지 수 추출하기
    result = prd_detail[0]
    page = result.text.split('|')[0]
    return page

# 2. 사용자 정의 함수 - 데이터프레임 데이터 도서의 쪽수를 한번에 반환하기
def get_multiple_page_cnt(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [19]:
# 테스트 : 함수 호출
get_page_cnt(9791189327156)

[<td class="txt lastCol">300쪽 | 480g | 135*210*20mm</td>]


'300쪽 '

In [20]:
# 테스트 : 함수 호출
## 인기 도서 10권 추출 - 데이터프레임
top10_books = books.head(10) 
top10_books
## 각 행의 반복 작업을 수행하기 위해 데이터프레임의 apply()메서드 적용
page_count = top10_books.apply(get_multiple_page_cnt, axis = 1)

# 전체 도서에 적용하면 ConnectionError 발생 가능
# page_count = books.apply(get_multiple_page_cnt, axis = 1)

AttributeError: 'list' object has no attribute 'head'

In [ ]:
# 쪽수 시리즈 생성
page_count = top10_books.apply(get_multiple_page_cnt, axis = 1)

type(page_count)
page_count.name = 'page_count'
print(page_count)

## 4. 기존 데이터셋과 추가 데이터셋 합치기  
* pandas.merge(df, series, left_index=, rught_index=)    

### [문제]  top10_books와 쪽수 시리즈 합치기 

In [ ]:
# 데이터프레임과 시리즈 합치기
top10_with_page_count = pd.merge(top10_books, page_count,
                                 left_index=True, right_index=True)
top10_with_page_count

## [연습] merge() 함수의 매개변수 이해하기

In [ ]:
df1 = pd.DataFrame({'col1': ['a','b','c'], 'col2': [1,2,3]})
df1

In [ ]:
df2 = pd.DataFrame({'col1': ['a','b','d'], 'col3': [10,20,30]})
df2

In [ ]:
#df1.merge(df2, on = 'col1')

In [ ]:
pd.merge(df1, df2, on='col1')


In [ ]:
pd.merge(df1, df2, how='left', on='col1')

In [ ]:
pd.merge(df1, df2, how='right', on='col1')

In [ ]:
pd.merge(df1, df2, how='outer', on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col1', right_on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col2', right_index=True)